In [2]:
!pip install minisom

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for minisom: filename=MiniSom-2.3.3-py3-none-any.whl size=11720 sha256=632bdc8ee7c58f1a2219b298f6936a395237fda45c77868bc7f62eb1ef623119
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\ec\7e\00\ac334ebb21e9016a73bef195e9732606217c6894f96bd51e6f
Successfully built minisom


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from minisom import MiniSom
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Load dataset
file_path = "data.csv"
df = pd.read_csv(file_path)



In [2]:
df.head()

,rs_CLEC3A,rs_CPB1,rs_SCGB2A2,rs_SCGB1D2,rs_TFF1,rs_MUCL1,rs_GSTM1,rs_PIP,rs_ADIPOQ,rs_ADH1B,...,pp_p27.pT198,pp_p38.MAPK,pp_p38.pT180.Y182,pp_p53,pp_p62.LCK.ligand,pp_p70S6K,pp_p70S6K.pT389,pp_p90RSK,pp_p90RSK.pT359.S363,vital.status
0,0.892818,6.580103,14.123672,10.606501,13.189237,6.649466,10.520335,10.338490,10.248379,10.229970,...,-0.043330,-0.002598,0.449228,-0.375230,-0.691766,-0.337863,-0.178503,0.011638,-0.207257,0
1,0.000000,3.691311,17.116090,15.517231,9.867616,9.691667,8.179522,7.911723,1.289598,1.818891,...,-0.220764,0.220809,1.035115,-0.074136,0.279067,0.292925,-0.155242,-0.089365,0.267530,0
2,3.748150,4.375255,9.658123,5.326983,12.109539,11.644307,10.517330,5.114925,11.975349,11.911437,...,0.010615,-0.133214,0.344969,-0.351936,0.219910,0.308110,-0.190794,-0.222150,-0.198518,0
3,0.000000,18.235519,18.535480,14.533584,14.078992,8.913760,10.557465,13.304434,8.205059,9.211476,...,0.064070,-0.384008,0.678042,0.096329,-0.266554,-0.079871,-0.463237,0.522998,-0.046902,0
4,0.000000,4.583724,15.711865,12.804521,8.881669,8.430028,12.964607,6.806517,4.294341,5.385714,...,-0.065488,0.209858,0.920408,0.042210,-0.441542,-0.152317,0.511386,-0.096482,0.037473,0


In [3]:
# Separate features and target variable
X = df.drop(columns=["vital.status"])
y = df["vital.status"]

In [4]:
# Normalize the data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

In [5]:
# Define dataset split
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)


In [6]:
# PCA Neural Network
pca_nn_model = Pipeline([
    ('pca', PCA(n_components=50)),
    ('classifier', LogisticRegression(random_state=42, max_iter=500))
])
pca_nn_model.fit(X_train, y_train)
y_pred_pca_nn = pca_nn_model.predict(X_test)
accuracy_pca_nn = accuracy_score(y_test, y_pred_pca_nn)
print(f'PCA Neural Network Accuracy: {accuracy_pca_nn:.4f}')


PCA Neural Network Accuracy: 0.9078


In [7]:
# Feedforward Neural Network (FFNN)
ffnn_model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu', solver='adam', max_iter=200, random_state=42)
ffnn_model.fit(X_train, y_train)
y_pred_ffnn = ffnn_model.predict(X_test)
accuracy_ffnn = accuracy_score(y_test, y_pred_ffnn)
print(f'FFNN Accuracy: {accuracy_ffnn:.4f}')

FFNN Accuracy: 0.8865


In [8]:
# Self-Organizing Map (SOM)
som_dim = (10, 10)
som = MiniSom(x=som_dim[0], y=som_dim[1], input_len=X_train.shape[1], sigma=1.0, learning_rate=0.5)
som.random_weights_init(X_train)
som.train_random(X_train, 1000)
winner_nodes = np.array([som.winner(x) for x in X_test])
som_labels = {tuple(node): label for node, label in zip(winner_nodes, y_test)}
y_pred_som = np.array([som_labels.get(tuple(node), 0) for node in winner_nodes])
accuracy_som = accuracy_score(y_test, y_pred_som)
print(f'SOM Accuracy: {accuracy_som:.4f}')

SOM Accuracy: 0.8865


In [9]:
# Deep Autoencoder
autoencoder = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(X_train.shape[1], activation='sigmoid')
])
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=16, validation_data=(X_test, X_test), verbose=1)
loss = autoencoder.evaluate(X_test, X_test)
print(f'Deep Autoencoder Loss: {loss:.4f}')

Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0618 - val_loss: 0.0362
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0370 - val_loss: 0.0323
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0336 - val_loss: 0.0303
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0313 - val_loss: 0.0292
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0301 - val_loss: 0.0282
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0287 - val_loss: 0.0274
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0275 - val_loss: 0.0263
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0268 - val_loss: 0.0254
Epoch 9/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0251 - val_loss: 0.0245
Epoch 10/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0244 - val_loss: 0.0240
Epoch 11/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0236 - val_loss: 0.0235
Epoch 12/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0231 - val_lo

In [10]:
# Compute accuracy for autoencoder reconstruction
y_pred_autoencoder = autoencoder.predict(X_test)
reconstruction_error = np.mean(np.abs(y_pred_autoencoder - X_test), axis=1)
threshold = np.percentile(reconstruction_error, 95)
y_pred_autoencoder_labels = (reconstruction_error < threshold).astype(int)
accuracy_autoencoder = accuracy_score(y_test, y_pred_autoencoder_labels)
print(f'Deep Autoencoder Accuracy: {accuracy_autoencoder:.4f}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Deep Autoencoder Accuracy: 0.1702


In [14]:
!pip install anfis

   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/920.8 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/920.8 kB ? eta -:--:--
   --------------------- ---------------- 524.3/920.8 kB 882.6 kB/s eta 0:00:01
   ---------------------------------------- 920.8/920.8 kB 1.1 MB/s eta 0:00:00


In [11]:
# Neuro-Fuzzy Hybrid Model using skfuzzy
import skfuzzy as fuzz
from skfuzzy import control as ctrl
input_var = ctrl.Antecedent(np.arange(0, 1, 0.01), 'input')
output_var = ctrl.Consequent(np.arange(0, 1, 0.01), 'output')
input_var.automf(3)
output_var.automf(3)
rule1 = ctrl.Rule(input_var['poor'], output_var['poor'])
rule2 = ctrl.Rule(input_var['average'], output_var['average'])
rule3 = ctrl.Rule(input_var['good'], output_var['good'])
control_system = ctrl.ControlSystem([rule1, rule2, rule3])
sim = ctrl.ControlSystemSimulation(control_system)
y_pred_fuzzy = []
for val in X_test[:, 0]:
    sim.input['input'] = val
    sim.compute()
    y_pred_fuzzy.append(sim.output['output'])
y_pred_fuzzy_labels = np.round(y_pred_fuzzy).astype(int)
accuracy_fuzzy = accuracy_score(y_test, y_pred_fuzzy_labels)
print(f'Neuro-Fuzzy Hybrid Accuracy: {accuracy_fuzzy:.4f}')

Neuro-Fuzzy Hybrid Accuracy: 0.7730


LSTM Model

In [16]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Load and preprocess the dataset
df = pd.read_csv('data.csv')  # Replace with actual dataset path
X = df.drop(columns=['vital.status'])  # Features
y = df['vital.status']  # Target

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data for LSTM input (samples, time steps, features)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # Fix applied here

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X.shape[1])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# Evaluate model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
accuracy_lstm = accuracy_score(y_test, y_pred)
print(f'LSTM Accuracy: {accuracy_lstm:.4f}')


Epoch 1/50


c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.8466 - loss: 0.4816 - val_accuracy: 0.8865 - val_loss: 0.3563
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8588 - loss: 0.4091 - val_accuracy: 0.8865 - val_loss: 0.3550
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8803 - loss: 0.3708 - val_accuracy: 0.8865 - val_loss: 0.3573
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8622 - loss: 0.4018 - val_accuracy: 0.8865 - val_loss: 0.3555
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8376 - loss: 0.4412 - val_accuracy: 0.8865 - val_loss: 0.3508
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8248 - loss: 0.4690 - val_accuracy: 0.8865 - val_loss: 0.3482
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8528 - loss: 0.4011 - val_accuracy: 0.8865 - val_loss: 0.3524
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8617 - loss: 0.3837 - val_accuracy: 0.8865 - val_loss: 0.3556
Ep

**After making the changes in ALl models**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, LSTM
from tensorflow.keras.optimizers import Adam

# Load dataset
data_path = "data.csv"
df = pd.read_csv(data_path)

# Apply PCA before feeding data to models
pca = PCA(n_components=50)
X_pca = pca.fit_transform(df.drop(columns=["vital.status"]))
y = df["vital.status"]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Modify FFNN to use Sigmoid activation instead of ReLU
ffnn_model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='logistic', solver='adam', max_iter=200, random_state=42)
ffnn_model.fit(X_train, y_train)
y_pred_ffnn = ffnn_model.predict(X_test)
accuracy_ffnn = accuracy_score(y_test, y_pred_ffnn)

# Improve Deep Autoencoder - Increase Layers, Use BatchNorm & Dropout, Take PCA Output as Input
autoencoder = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(512, activation='sigmoid'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(256, activation='sigmoid'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='sigmoid'),
    BatchNormalization(),
    Dense(256, activation='sigmoid'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(512, activation='sigmoid'),
    BatchNormalization(),
    Dense(X_train.shape[1], activation='sigmoid')  # Output layer remains Sigmoid
])

autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the improved autoencoder
autoencoder.fit(X_train, X_train, epochs=150, batch_size=16, validation_data=(X_test, X_test), verbose=1)

# Evaluate autoencoder performance
loss = autoencoder.evaluate(X_test, X_test)

# Modify LSTM Model to Use Sigmoid Activation
data_scaled = MinMaxScaler().fit_transform(df.drop(columns=["vital.status"]))
data_scaled = data_scaled.reshape((data_scaled.shape[0], 1, data_scaled.shape[1]))
X_train, X_test, y_train, y_test = train_test_split(data_scaled, y, test_size=0.2, random_state=42)

lstm_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# Evaluate LSTM Model
y_pred_lstm = (lstm_model.predict(X_test) > 0.5).astype(int)
accuracy_lstm = accuracy_score(y_test, y_pred_lstm)

# Results Table
results = pd.DataFrame({
    "Model": ["FFNN (Sigmoid)", "Deep Autoencoder (PCA Input)", "LSTM (Sigmoid)"],
    "Accuracy/Loss": [accuracy_ffnn, loss, accuracy_lstm],
    "Complexity": ["Moderate", "High", "High"]
})

# Display results
print(results)


c:\Users\DELL\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Epoch 1/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 74.8297 - val_loss: 72.4941
Epoch 2/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 74.8097 - val_loss: 72.3732
Epoch 3/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 75.0621 - val_loss: 72.1297
Epoch 4/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 74.8089 - val_loss: 71.7076
Epoch 5/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 75.8512 - val_loss: 71.0187
Epoch 6/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 72.7989 - val_loss: 70.2731
Epoch 7/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 74.9329 - val_loss: 69.5969
Epoch 8/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 71.4666 - val_loss: 69.0854
Epoch 9/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 73.5832 - val_loss: 68.7807
Epoch 10/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 70.1805 - val_loss: 68.5560
Epoch 11/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 73.1335 - val_loss: 68.4478
Epoch 12/150
36/36 ━━━━━━━━━━━━━━━━━━━━ 

c:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.7942 - loss: 0.4894 - val_accuracy: 0.8865 - val_loss: 0.3539
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8595 - loss: 0.4092 - val_accuracy: 0.8865 - val_loss: 0.3541
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8313 - loss: 0.4634 - val_accuracy: 0.8865 - val_loss: 0.3534
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8705 - loss: 0.3821 - val_accuracy: 0.8865 - val_loss: 0.3576
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8663 - loss: 0.3949 - val_accuracy: 0.8865 - val_loss: 0.3557
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8371 - loss: 0.4453 - val_accuracy: 0.8865 - val_loss: 0.3529
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8541 - loss: 0.4167 - val_accuracy: 0.8865 - val_loss: 0.3535
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8541 - loss: 0.4028 - val_accuracy: 0.8865 - val_loss: 0.3532
Ep